In [1]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/mental/mental-bert-base-uncased

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/mental/mental-bert-base-uncased)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [3]:
from huggingface_hub import login
login(new_session=False)

In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="mental/mental-bert-base-uncased")

config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("mental/mental-bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("mental/mental-bert-base-uncased")

In [11]:
!pip install -q transformers datasets accelerate scikit-learn

import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import torch
from torch.utils.data import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

In [12]:
with open("twitter_sample.json", "r") as f:
    raw = json.load(f)

df = pd.DataFrame(raw)

# We care about the text and the disorder label
df = df[["text", "disorder"]].dropna()

print(df.head())
print("Unique disorders:", df["disorder"].unique())

# 2. Create label mapping (string -> int)
labels = sorted(df["disorder"].unique().tolist())
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}
print("label2id:", label2id)

df["label"] = df["disorder"].map(label2id)


                                                text disorder
0  When the fistful of edibles hit and I’m on my ...     adhd
1                            https://t.co/xOVc67kEqo     adhd
2  Me after getting smacked through a fuckin wall...     adhd
3                       Hill Country Barbecue Market     adhd
4  My one contribution to 2020 in the form of a B...     adhd
Unique disorders: ['adhd' 'anxiety' 'bipolar' 'depression']
label2id: {'adhd': 0, 'anxiety': 1, 'bipolar': 2, 'depression': 3}


In [13]:
X_train, X_temp, y_train, y_temp = train_test_split(
    df["text"],
    df["label"],
    test_size=0.2,
    random_state=42,
    stratify=df["label"],
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp,
)

print("Train size:", len(X_train))
print("Val size:", len(X_val))
print("Test size:", len(X_test))

Train size: 2992
Val size: 374
Test size: 375


In [14]:
model_name = "mental/mental-bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

num_labels = len(labels)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = list(texts)
        self.labels = list(labels)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        encodings = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
        )

        item = {
            "input_ids": torch.tensor(encodings["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(encodings["attention_mask"], dtype=torch.long),
            "labels": torch.tensor(label, dtype=torch.long),
        }
        return item

train_dataset = TweetDataset(X_train, y_train, tokenizer)
val_dataset   = TweetDataset(X_val,   y_val,   tokenizer)
test_dataset  = TweetDataset(X_test,  y_test,  tokenizer)


In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)

    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels,
        preds,
        average="macro",
        zero_division=0,
    )

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [17]:
training_args = TrainingArguments(
    output_dir="./mentalbert_twitter_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to=[],              # <- no wandb
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [18]:
trainer.train()

test_metrics = trainer.evaluate(test_dataset)
print("Test metrics:", test_metrics)

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.986300,0.922010,0.652406,0.751416,0.504338,0.528562
2,0.756900,0.821372,0.692513,0.669059,0.575751,0.598090
3,0.577300,0.828103,0.689840,0.661153,0.592670,0.610355


Test metrics: {'eval_loss': 0.824857234954834, 'eval_accuracy': 0.672, 'eval_precision': 0.6330334345040227, 'eval_recall': 0.54632150636249, 'eval_f1': 0.5584013550449368, 'eval_runtime': 2.4989, 'eval_samples_per_second': 150.067, 'eval_steps_per_second': 4.802, 'epoch': 3.0}
